In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import matplotlib.pyplot as plt
import time
from os import listdir
import random
from PIL import Image, ImageOps

#### Parameters

In [2]:
STILL_ALIVE_REWARD = 1

CROP_SHAPE = (750, 539, 1)

#### Browser functions

In [3]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard():        
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    board.click()
    
    
def getScreen():
    global index
    fileName = 'data/shots/screen' + str(index) + '.png'
    print(fileName)
    
    state = getState()
    if state == 'playing' or state == 'paused':
    
        ss = browser.get_screenshot_as_file(fileName)

        index += 1

    return fileName
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def makeMove(m):
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    if m == 1:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == 2:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == 3:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == 4:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

def makeRandomMove():
    r = random.randint(0, 4)
    makeMove(r)
    
    return r

#### Preprocess functions

In [4]:
folder = 'data/shots/'


def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)
        

def preprocImg(fileName):
    '''Preprocess a screenshot.'''
    im = Image.open(fileName)    
    
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    im.save(fileName)
    
    
def preprocAll():
    '''Preprocess all screenshots in the data folder.'''
    files = listdir(folder)
    for f in files:
        preprocImg(folder + f)

#### Play game

In [6]:
url = 'https://playsnake.org/'

browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
# browser = webdriver.Chrome(executable_path='D:/Biblioteci/Python/chromedriver_win32/chromedriver.exe')
browser.get(url) 


index = 0
history = []

maxGames = 2

for g in range(maxGames):
    
    # Select the level
    selectLevel(1)
    time.sleep(2)
    
    score = 0
    
#     while getState() == 'playing':
#         clickBoard()
#         s1 = getScreen()
#         clickBoard()
        
#         clickBoard()
#         s2 = getScreen()
#         score = getScore()

#         clickBoard()
#         m = makeRandomMove()

#         clickBoard()
#         s3 = getScreen()
#         r = max(1, getScore() - score)

#         clickBoard()
#         history.append([g, s1, s2, r, m, s3])
#         score += r

    
    while getState() == 'playing':
        clickBoard()
        s = getScreen()

        clickBoard()
        m = makeRandomMove()

        clickBoard()
        s2 = getScreen()
        r = max(STILL_ALIVE_REWARD, getScore() - score)
        history.append([g, index, m, r, score])
        score = getScore()
        
        clickBoard()
        
    clickBoard()
    
print(history)

browser.quit()


data/shots/screen0.png
data/shots/screen1.png
data/shots/screen2.png
data/shots/screen3.png
data/shots/screen4.png
data/shots/screen5.png
data/shots/screen6.png
data/shots/screen7.png
data/shots/screen8.png
data/shots/screen9.png
data/shots/screen10.png
data/shots/screen11.png


WebDriverException: Message: unknown error: Element <div class="board">...</div> is not clickable at point (518, 312). Other element would receive the click: <nav>...</nav>
  (Session info: chrome=72.0.3626.119)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.10586 x86_64)


#### Q model

In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses

Using TensorFlow backend.


Intai fac un model care doar prezice daca un board reprezinta un joc care s-a terminat (lovit de zid) sau nu

In [19]:
def readData():
    files = listdir(folder)
    for f in files:
        preprocImg(folder + f)


def nn_model():
    model = Sequential()
    
    model.add(Conv2D(32, (5, 5), input_shape=CROP_SHAPE, activation='relu', name='conv1'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool1'))
    
    model.add(Conv2D(32, (5, 5), activation='relu', name='conv3'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool2'))
    
    model.add(Flatten(name='flat'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dense(1, activation='softmax', name='dense2'))
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam')
    
    return model
    
model = nn_model()
print(model)